**Parallel Programming using CUDA C**

**Problem Statement 1: Vector Addition using CUDA**

**Problem Statement:**

**Write a CUDA C program that performs element-wise addition of two vectors A and B of size N. The result of the addition should be stored in vector C.**

Details:
1. Initialize the vectors A and B with random numbers.
2. The output vector C[i] = A[i] + B[i], where i ranges from 0 to N-1.
3. Use CUDA kernels to perform the computation in parallel.
4. Write the code for both serial (CPU-based) and parallel (CUDA-based) implementations.
5. Measure the execution time of both the serial and CUDA implementations for different values of N (e.g., N = 10^5, 10^6, 10^7).

Task:

   • Calculate and report the speedup (i.e., the ratio of CPU execution time to GPU execution time).

In [1]:
%%writefile vector_add_cuda.cu

#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>
#include <time.h>

// CUDA kernel for vector addition
__global__ void vectorAddCUDA(float *A, float *B, float *C, int N) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < N) {
        C[i] = A[i] + B[i];
    }
}

int main(int argc, char *argv[]) {
    if (argc != 2) {
        fprintf(stderr, "Usage: %s <vector_size>\n", argv[0]);
        return 1;
    }

    int N = atoi(argv[1]);  // Get vector size from command line argument
    if (N <= 0) {
        fprintf(stderr, "Error: vector size must be a positive integer.\n");
        return 1;
    }

    size_t size = N * sizeof(float);

    // Allocate memory on host
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    // Initialize vectors with random values
    srand(time(NULL));  // Seed the random number generator
    for (int i = 0; i < N; i++) {
        h_A[i] = rand() % 100;
        h_B[i] = rand() % 100;
    }

    // Allocate memory on device (GPU)
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // Copy vectors from host to device
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Measure time for GPU execution
    clock_t start = clock();

    // Define grid and block sizes
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;

    // Launch kernel
    vectorAddCUDA<<<numBlocks, blockSize>>>(d_A, d_B, d_C, N);

    // Synchronize the device
    cudaDeviceSynchronize();

    clock_t end = clock();

    // GPU execution time
    double gpu_time = (double)(end - start) / CLOCKS_PER_SEC;
    printf("GPU execution time: %f seconds\n", gpu_time);

    // Copy result from device to host
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

Overwriting vector_add_cuda.cu


In [2]:
!nvcc vector_add_cuda.cu -o vector_add_cuda

In [3]:
!./vector_add_cuda 100000

GPU execution time: 0.000201 seconds


In [4]:
!./vector_add_cuda 1000000

GPU execution time: 0.000242 seconds


In [5]:
!./vector_add_cuda 10000000

GPU execution time: 0.000649 seconds


**Problem Statement 2: Matrix Addition using CUDA**

**Problem Statement:**

**Write a CUDA C program to perform element-wise addition of two matrices A and B of size M x N. The result of the addition should be stored in matrix C.**

Details:
.
1.   Initialize the matrices A and B with random values
2.   The output matrix C[i][j] = A[i][j] + B[i][j] where i ranges from 0 to M-1 and j ranges from 0 to N-1.
3.   Write code for both serial (CPU-based) and parallel (CUDA-based) implementations.
4.   Measure the execution time of both implementations for various matrix sizes (e.g., 100x100, 500x500, 1000x1000).

Task:

• Calculate the speedup using the execution times of the CPU and GPU implementations.

In [11]:
%%writefile matrix_add_cuda.cu

#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>
#include <time.h>

// CUDA kernel for matrix addition
__global__ void matrixAddCUDA(float *A, float *B, float *C, int M, int N) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < M && j < N) {
        int index = i * N + j;
        C[index] = A[index] + B[index];
    }
}

// Helper function to allocate a 2D array on the device
float* allocate2DArrayOnDevice(int M, int N) {
    float *array;
    cudaMalloc((void**)&array, M * N * sizeof(float));
    return array;
}

// Helper function to free a 2D array on the device
void free2DArrayOnDevice(float *array) {
    cudaFree(array);
}

int main(int argc, char *argv[]) {
    if (argc != 3) {
        fprintf(stderr, "Usage: %s <rows> <columns>\n", argv[0]);
        return 1;
    }

    int M = atoi(argv[1]);  // Get number of rows from command line argument
    int N = atoi(argv[2]);  // Get number of columns from command line argument
    if (M <= 0 || N <= 0) {
        fprintf(stderr, "Error: both dimensions must be positive integers.\n");
        return 1;
    }

    // Allocate memory on host
    float *h_A = (float *)malloc(M * N * sizeof(float));
    float *h_B = (float *)malloc(M * N * sizeof(float));
    float *h_C = (float *)malloc(M * N * sizeof(float));

    // Initialize matrices with random values
    srand(time(NULL));  // Seed the random number generator
    for (int i = 0; i < M * N; i++) {
        h_A[i] = rand() % 100;
        h_B[i] = rand() % 100;
    }

    // Allocate memory on device (GPU)
    float *d_A = allocate2DArrayOnDevice(M, N);
    float *d_B = allocate2DArrayOnDevice(M, N);
    float *d_C = allocate2DArrayOnDevice(M, N);

    // Copy matrices from host to device
    cudaMemcpy(d_A, h_A, M * N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, M * N * sizeof(float), cudaMemcpyHostToDevice);

    // Define grid and block sizes
    dim3 threadsPerBlock(4, 4);  // 16x16 block size
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (M + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Measure time for GPU execution
    clock_t start = clock();

    // Launch kernel
    matrixAddCUDA<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, M, N);

    // Synchronize the device
    cudaDeviceSynchronize();

    clock_t end = clock();

    // GPU execution time
    double gpu_time = (double)(end - start) / CLOCKS_PER_SEC;
    printf("GPU execution time: %f seconds\n", gpu_time);

    // Copy result from device to host
    cudaMemcpy(h_C, d_C, M * N * sizeof(float), cudaMemcpyDeviceToHost);

    // Print a few elements of the result matrix
    // printf("Result matrix C (first 5 elements):\n");
    // for (int i = 0; i < 5 && i < M * N; i++) {
    //    printf("%f ", h_C[i]);
    // }
    // printf("\n");

    // Free device memory
    free2DArrayOnDevice(d_A);
    free2DArrayOnDevice(d_B);
    free2DArrayOnDevice(d_C);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

Overwriting matrix_add_cuda.cu


In [12]:
!nvcc matrix_add_cuda.cu -o matrix_add_cuda

In [13]:
!./matrix_add_cuda 100 100

GPU execution time: 0.000184 seconds


In [14]:
!./matrix_add_cuda 500 500

GPU execution time: 0.000224 seconds


In [15]:
!./matrix_add_cuda 1000 1000

GPU execution time: 0.000409 seconds


**Problem Statement 3: Dot Product of Two Vectors using CUDA**

**Problem Statement:**

**Write a CUDA C program to compute the dot product of two vectors A and B of size N. The dot product is defined as:**

Details:
1. Initialize the vectors A and B with random values.
2. Implement the dot product calculation using both serial (CPU) and parallel (CUDA) approaches.
3. Measure the execution time for both implementations with different vector sizes (e.g., N = 10^5, 10^6, 10^7).
4. Use atomic operations or shared memory reduction in the CUDA kernel to compute the final sum.
      
Task:

• Calculate and report the speedup for different vector sizes.

In [16]:
%%writefile dot_product_cuda.cu

#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>
#include <time.h>

// CUDA kernel for dot product using atomic operations
__global__ void dotProductCUDA(float *A, float *B, float *C, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        atomicAdd(C, A[idx] * B[idx]);
    }
}

// Serial dot product calculation for CPU
float dotProductCPU(float *A, float *B, int N) {
    float sum = 0;
    for (int i = 0; i < N; i++) {
        sum += A[i] * B[i];
    }
    return sum;
}

int main(int argc, char *argv[]) {
    if (argc != 2) {
        fprintf(stderr, "Usage: %s <vector_size>\n", argv[0]);
        return 1;
    }

    int N = atoi(argv[1]);  // Get vector size from command line argument
    if (N <= 0) {
        fprintf(stderr, "Error: vector size must be a positive integer.\n");
        return 1;
    }

    size_t size = N * sizeof(float);

    // Allocate memory on host
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(sizeof(float));

    // Initialize vectors with random values
    srand(time(NULL));  // Seed the random number generator
    for (int i = 0; i < N; i++) {
        h_A[i] = rand() % 100;
        h_B[i] = rand() % 100;
    }
    *h_C = 0;  // Initialize result to 0

    // Allocate memory on device (GPU)
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, sizeof(float));

    // Copy vectors from host to device
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C, h_C, sizeof(float), cudaMemcpyHostToDevice);

    // Define grid and block sizes
    int blockSize = 256;  // 256 threads per block
    int gridSize = (N + blockSize - 1) / blockSize;

    // Measure time for GPU execution
    clock_t start = clock();

    // Launch kernel
    dotProductCUDA<<<gridSize, blockSize>>>(d_A, d_B, d_C, N);

    // Synchronize the device
    cudaDeviceSynchronize();

    clock_t end = clock();

    // GPU execution time
    double gpu_time = (double)(end - start) / CLOCKS_PER_SEC;

    // Copy result from device to host
    cudaMemcpy(h_C, d_C, sizeof(float), cudaMemcpyDeviceToHost);

    printf("GPU Dot Product: %f\n", *h_C);
    printf("GPU execution time: %f seconds\n", gpu_time);

    // Calculate speedup (assuming CPU result calculation is not included in this code)
    // Uncomment below if CPU timing is added
    // double speedup = cpu_time / gpu_time;
    // printf("Speedup (CPU/GPU): %f\n", speedup);

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

Writing dot_product_cuda.cu


In [17]:
!nvcc dot_product_cuda.cu -o dot_product_cuda

In [18]:
!./dot_product_cuda 100000

GPU Dot Product: 245451424.000000
GPU execution time: 0.055302 seconds


In [19]:
!./dot_product_cuda 1000000

GPU Dot Product: 2450514432.000000
GPU execution time: 0.003724 seconds


In [20]:
!./dot_product_cuda 10000000

GPU Dot Product: 24211341312.000000
GPU execution time: 0.035262 seconds


**Problem Statement 4: Matrix Multiplication using CUDA**

**Problem Statement:**

**Write a CUDA C program to perform matrix multiplication. Given two matrices A (MxN) and B (NxP), compute the resulting matrix C (MxP) where:**

Details:
1. Initialize the matrices A and B with random values.
2. Write code for both serial (CPU-based) and parallel (CUDA-based) implementations.
3. Measure the execution time of both implementations for various matrix sizes (e.g., 100x100, 500x500, 1000x1000).

Task:

• Calculate the speedup by comparing the CPU and GPU execution times.

In [21]:
%%writefile matrix_multiplication_cuda.cu

#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>
#include <time.h>

// CUDA kernel for matrix multiplication
__global__ void matrixMultiplicationCUDA(float *A, float *B, float *C, int M, int N, int P) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < P) {
        float sum = 0;
        for (int k = 0; k < N; k++) {
            sum += A[row * N + k] * B[k * P + col];
        }
        C[row * P + col] = sum;
    }
}

int main(int argc, char *argv[]) {
    if (argc != 4) {
        fprintf(stderr, "Usage: %s <M> <N> <P>\n", argv[0]);
        return 1;
    }

    int M = atoi(argv[1]);  // Rows of A and C
    int N = atoi(argv[2]);  // Columns of A and Rows of B
    int P = atoi(argv[3]);  // Columns of B and C

    if (M <= 0 || N <= 0 || P <= 0) {
        fprintf(stderr, "Error: All dimensions must be positive integers.\n");
        return 1;
    }

    size_t size_A = M * N * sizeof(float);
    size_t size_B = N * P * sizeof(float);
    size_t size_C = M * P * sizeof(float);

    // Allocate memory on host
    float *h_A = (float *)malloc(size_A);
    float *h_B = (float *)malloc(size_B);
    float *h_C = (float *)malloc(size_C);

    // Initialize matrices A and B with random values
    srand(time(NULL));  // Seed the random number generator
    for (int i = 0; i < M * N; i++) {
        h_A[i] = rand() % 100;
    }
    for (int i = 0; i < N * P; i++) {
        h_B[i] = rand() % 100;
    }

    // Allocate memory on device (GPU)
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size_A);
    cudaMalloc(&d_B, size_B);
    cudaMalloc(&d_C, size_C);

    // Copy matrices from host to device
    cudaMemcpy(d_A, h_A, size_A, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size_B, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 blockSize(16, 16);  // Block of 16x16 threads
    dim3 gridSize((P + blockSize.x - 1) / blockSize.x, (M + blockSize.y - 1) / blockSize.y);

    // Measure GPU execution time
    clock_t start = clock();

    // Launch CUDA kernel
    matrixMultiplicationCUDA<<<gridSize, blockSize>>>(d_A, d_B, d_C, M, N, P);

    // Synchronize device
    cudaDeviceSynchronize();

    clock_t end = clock();

    double gpu_time = (double)(end - start) / CLOCKS_PER_SEC;
    printf("GPU execution time: %f seconds\n", gpu_time);

    // Copy result from device to host
    cudaMemcpy(h_C, d_C, size_C, cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

Writing matrix_multiplication_cuda.cu


In [22]:
!nvcc matrix_multiplication_cuda.cu -o matrix_multiplication_cuda

In [24]:
!./matrix_multiplication_cuda 100 100 100

GPU execution time: 0.043653 seconds


In [25]:
!./matrix_multiplication_cuda 500 500 500

GPU execution time: 0.001274 seconds


In [26]:
!./matrix_multiplication_cuda 1000 1000 1000

GPU execution time: 0.007127 seconds
